In [ ]:
######SEGMENTATION CH02, ONLY OUTSIDE AND CELL BODY WITH LABEL ##################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: segmentation (CELL vs OUTSIDE)
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.4*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(3))

# CELL (soma) via distance transform
dist = edt(neuron_mask)
cell_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# --------------------------
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# --------------------------
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# --------------------------
# 6) Visualization (CELL vs OUTSIDE only)
# --------------------------
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
    colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        name='Lysosomes (cell vs outside)'
    )
    try:
        pts.face_color = colors
        pts.edge_color = 'black'
        pts.edge_width = 0.3
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy(),
            'cell':   df['cell_id_ch2'].to_numpy()
        }
        # Show BOTH IDs on each lysosome
        pts.text = {'text': 'ID_L:{lys_id}  ID_C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if len(df) > 0 and n_cells > 0 and body_lab.max() > 0:
        # cells with lysosomes inside 'cell' mask
        cell_hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
        ids_with_lyso = set(np.unique(cell_hits["cell_id_ch2"].to_numpy()))
        all_ids = set(range(1, int(body_lab.max()) + 1))
        ids_without_lyso = all_ids - ids_with_lyso

        props = regionprops(body_lab)
        pts_yes, labels_yes = [], []
        pts_no, labels_no   = [], []

        for prop in props:
            cid = int(prop.label)
            cz, cy, cx = prop.centroid  # (z, y, x)
            if cid in ids_with_lyso:
                pts_yes.append([cz, cy, cx])
                labels_yes.append(f"Cell {cid}")
            else:
                pts_no.append([cz, cy, cx])
                labels_no.append(f"Cell {cid}")

        # layer: cell labels (HAS lysosomes) - yellow
        if pts_yes:
            lbl_yes = viewer.add_points(
                np.asarray(pts_yes, dtype=float),
                name="Cell labels (HAS lysosomes)",
                size=0.1
            )
            lbl_yes.face_color = [0, 0, 0, 0]
            lbl_yes.edge_color = [0, 0, 0, 0]
            lbl_yes.edge_width = 0
            lbl_yes.text = {"text": labels_yes, "size": 28, "color": "yellow", "anchor": "center"}
            lbl_yes.blending = "translucent_no_depth"

        # layer: cell labels (NO lysosomes) - light gray
        if pts_no:
            lbl_no = viewer.add_points(
                np.asarray(pts_no, dtype=float),
                name="Cell labels (NO lysosomes)",
                size=0.1
            )
            lbl_no.face_color = [0, 0, 0, 0]
            lbl_no.edge_color = [0, 0, 0, 0]
            lbl_no.edge_width = 0
            lbl_no.text = {"text": labels_no, "size": 28, "color": "lightgray", "anchor": "center"}
            lbl_no.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# --------------------------
# 8) Save a 2D XY screenshot (post-segmentation)
# --------------------------
def _best_z_index_for_xy(mask_zyx: np.ndarray) -> int:
    """Pick the Z slice with the largest 'cell' area; fallback to middle slice."""
    if mask_zyx.ndim != 3:
        return 0
    areas = [int(mask_zyx[z].sum()) for z in range(mask_zyx.shape[0])]
    z_best = int(np.argmax(areas))
    if areas[z_best] == 0:
        z_best = mask_zyx.shape[0] // 2
    return z_best

def save_xy_2d_screenshot(viewer, z_index=None, path='ch2_segmentation_XY_2d.png'):
    """Save a 2D screenshot at a single Z-plane (XY view)."""
    viewer.dims.ndisplay = 2
    # choose Z slice
    if z_index is None:
        z_index = _best_z_index_for_xy(cell_mask)
    try:
        viewer.dims.set_current_step(0, int(z_index))  # axis 0 is Z
    except Exception:
        # older napari fallback
        try:
            cs = list(viewer.dims.current_step)
            cs[0] = int(z_index)
            viewer.dims.current_step = tuple(cs)
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 2D XY screenshot (z={z_index}): {path}")

# Set title + save a 2D XY shot (best Z automatically)
try:
    viewer.title = "CH2 segmentation: cell vs outside (2D XY snapshot)"
except Exception:
    pass
save_xy_2d_screenshot(viewer, path='ch2_segmentation_XY_2d.png')

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()